In [2]:
pip install yfinance

     |████████████████████████████████| 5.5MB 6.3MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.56-py2.py3-none-any.whl size=22627 sha256=a9fa56646bb09bef511de2d4445f21ce6f6c74087c5a81d24015beb66cc0313c
  Stored in directory: /root/.cache/pip/wheels/e9/26/69/ebde3c87490e75c5d8b06cf2dd284d1183ca1bb814b1d5f18c
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [3]:
import pandas as pd
import requests
import json
import ast
import yaml
import urllib
import yfinance
from datetime import datetime, timedelta
#from google.colab import drive

In [4]:

df_tweets = pd.read_csv('https://raw.githubusercontent.com/ml-group8/assignment/main/df_tweets_sentiment.csv')
# Keep only required columns
df_tweets = df_tweets[['created_at', 'id', 'tweet', 'sentiment', 'confidence']]

# Convert created_at column fro mstring to datetime
def string_to_datetime(str_date):
  str_date = str_date[:19]
  return datetime.strptime(str_date, "%Y-%m-%dT%H:%M:%S")

df_tweets['created_at'] = df_tweets['created_at'].map(lambda strDateTime: string_to_datetime(strDateTime))

max_date = df_tweets['created_at'].max()
min_date = df_tweets['created_at'].min()
type(min_date)

pandas._libs.tslibs.timestamps.Timestamp

In [26]:
from datetime import datetime, timedelta, timezone
#print(max_date.strftime('%Y-%m-%d'))

#TODO adjust to UTC as the tweets are at that timezone

#A/p the yfinance docs, Note: yahoo finance datetimes are received as UTC.
#however when visualizing the df the Datetime field is UTC-4 ???
drive_etf_ticker = yfinance.Ticker("DRIV")
drive_etf = drive_etf_ticker.history(
    start=min_date.strftime('%Y-%m-%d'),
    end=max_date.strftime('%Y-%m-%d'),
    interval='60m'
).reset_index()

# The Yahoo finance are in EST so convert them to UTC to match the tweets' timezone
drive_etf['DatetimeUTC'] = drive_etf['Datetime'].dt.tz_convert('UTC')
# Check DatetimeUTC really is UTC
drive_etf

,Datetime,Open,High,Low,Close,Volume,Dividends,Stock Splits,DatetimeUTC
0,2021-03-15 09:30:00-04:00,27.338800,27.450001,27.260000,27.389999,161430,0,0,2021-03-15 13:30:00+00:00
1,2021-03-15 10:30:00-04:00,27.409201,27.450001,27.270000,27.270000,47716,0,0,2021-03-15 14:30:00+00:00
2,2021-03-15 11:30:00-04:00,27.264400,27.320000,27.180000,27.299999,47115,0,0,2021-03-15 15:30:00+00:00
3,2021-03-15 12:30:00-04:00,27.306000,27.349501,27.299999,27.329000,26073,0,0,2021-03-15 16:30:00+00:00
4,2021-03-15 13:30:00-04:00,27.330000,27.429800,27.309999,27.419901,47720,0,0,2021-03-15 17:30:00+00:00
5,2021-03-15 14:30:00-04:00,27.429800,27.445900,27.380100,27.389999,34435,0,0,2021-03-15 18:30:00+00:00
6,2021-03-15 15:30:00-04:00,27.389999,27.500000,27.379999,27.495001,119392,0,0,2021-03-15 19:30:00+00:00
7,2021-03-16 09:30:00-04:00,27.549999,27.670000,27.460100,27.610001,195642,0,0,2021-03-16 13:30:00+00:00
8,2021-03-16 10:30:00-04:00,27.629999,27.674999,27.530001,27.594999,80474,0,0,2021-03-16 14:30:00+00:00
9,2021-03-16 11:30:00-04:00,27.600000,27.639900,27.420099,27.445000,109627,0,0,2021-03-16 15:30:00+00:00


In [ ]:

del drive_etf['Datetime']